# 0. Carga de librerías

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
import pandas as pd

# 1. Obtención del contenido de la web

Como la web está en Javascript, hemos empleado Selenium para poder obtener el contenido de la url. Para ello hemos instalado un driver para chrome. De esta manera ya si hemos podido utilizar la librería BeautifulSoup para obtener el contenido de la página web.

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
url = "https://www4.unfccc.int/sites/NDCStaging/Pages/All.aspx"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/concharomansantos/.wdm/drivers/chromedriver/mac64/95.0.4638.54/chromedriver] found in cache


A continuación una alternativa al bloque anterior 

In [3]:
#import chromedriver_binary 
#url = "https://www4.unfccc.int/sites/NDCStaging/Pages/All.aspx"
#driver = webdriver.Chrome()
#driver.get(url)
#soup = BeautifulSoup(driver.page_source, 'html.parser')
#driver.close()

# 2. Web Scraping

El código de la web se disponía en múltiples divs, que a su vez estaban formados por múltiples divs, y así sucesivamente. Por tanto, aunque no es un código muy óptimo, recorrer el código con bucles for es la única manera que se nos ocurría para poder obtener los datos e información de interés. Cada campo de interés lo hemos almacenado en una lista y al finalizar el recorrido por todo el código formamos el dataframe incluyendo en cada campo los valores contenidos en cada lista.

In [4]:
country_list = []
pdf_list = []
language_list = []
date_list = []
for letter_panel in soup.find_all(class_="panel-group letter-panel"):
    for panel_tag in letter_panel.find_all(class_="panel panel-default"):
        for x in panel_tag.find_all(class_='panel-collapse collapse'):
            for row in x.find_all(class_='row', style=True):
                for country in row.find_all(class_='col-sm-5'):
                    if (country.text != "Title"):
                        country_list.append(country.text) 
                for pdf in row.find_all('a', target=True, href=True):
                    pdf_list.append(pdf['href'])
                for language in row.find_all(class_='col-sm-2'):
                    if ((language.text != "Language") and (language.text != 'NDC') 
                        and (language.text != '') and (language.text != 'Translation')
                       and (language.text != 'Addendum')):
                        language_list.append(language.text)
                for date in row.find_all(class_='col-sm-3'):
                    if (date.text != "Submission date"):
                        date_list.append(date.text)

# 3. Creación del dataset

Creamos un dataframe pandas.

In [5]:
df = pd.DataFrame({'Version': country_list, 'PDF': pdf_list, 'Language': language_list, 'Date': date_list})

# 4. Procesado y limpieza del dataset

Eliminamos los espacios en blanco del campo "Version".

In [6]:
df['Version'] = df['Version'].str.strip()

Como el campo "Version" contiene el nombre del país, y el nombre de versión, queremos obtener un campo que solo contenga el nombre del país y otro para el nombre de versión. Para ello separamos el string del campo "Version" por el substring "NDC" y nos quedamos con la primera parte. Y de esa parte volvemos a separar por " " para el nombre de versión del nombre del país.

In [7]:
df['temp'] = df['Version'].str.split('NDC').str[0]
df['temp2'] = df['temp'].str.split(' ').str[:-2]
df['Country'] = [' '.join(map(str, l)) for l in df['temp2']]
ord_numbers = ['First', 'Second', 'Third', 'Fourth', 'Fifth']
for number in ord_numbers:
    df.loc[df.Version.str.contains(number), "Version"] = number + df['Version'].str.split('NDC').str[1]

Limpiamos algunos registros que contenían información en distinto formato, pero con los cuales no perdemos información.

In [8]:
df = df[df['Country'] != '']
df = df[df['Version'].str.contains('|'.join(ord_numbers),regex=True)]

En el NDC_Registry hay un registro por cada país de la European Union y otro de la Union en su conjunto, pero al inspeccionar los documentos encontramos que se repiten pues sólo hay un único NDC (y sus correspondientes versiones) para todos los países de la Unión. Por ello hemos optado por eliminar todos los registros de los países individuales (que por la misma razón señalada y las características de nuestra búsqueda se muestran con la etiqueta "EU") y dejar sólo el de la Unión en su conjunto, que se muestra con la etiqueta "European Union". 

In [9]:
df = df[df['Country'] != 'EU']

 De esta manera en nuestro dataframe tenemos los siguientes campos:
+ **"Country":** contiene el nombre del país.
+ **"Version":** contiene el nombre de la versión del fichero.
+ **"Language":** contiene el idioma en el que está escrito el NDC.
+ **"Date":** contiene la fecha en la que se publicó el fichero NDC.
+ **"PDF":** contiene la url del fichero NDC.

In [10]:
df = df[['Country','Version', 'Language', 'Date', 'PDF']]

Mostramos una muestra de registros de nuestro dataset final.

In [11]:
df.head(30)

,Country,Version,Language,Date,PDF
0,Afghanistan,First,English,23/11/2016,https://www4.unfccc.int/sites/ndcstaging/Publi...
1,Albania,First (Archived),English,21/09/2016,https://www4.unfccc.int/sites/ndcstaging/Publi...
2,Albania,First (Updated submission),English,12/10/2021,https://www4.unfccc.int/sites/ndcstaging/Publi...
3,Algeria,First,French,20/10/2016,https://www4.unfccc.int/sites/ndcstaging/Publi...
4,Algeria,First Translation,English,20/10/2016,https://www4.unfccc.int/sites/ndcstaging/Publi...
5,Andorra,First (Archived),French,24/03/2017,https://www4.unfccc.int/sites/ndcstaging/Publi...
6,Andorra,First (Updated submission),Spanish,20/05/2020,https://www4.unfccc.int/sites/ndcstaging/Publi...
7,Angola,First (Archived),English,16/11/2020,https://www4.unfccc.int/sites/ndcstaging/Publi...
8,Angola,First (Updated submission),English,31/05/2021,https://www4.unfccc.int/sites/ndcstaging/Publi...
9,Antigua and Barbuda,First (Archived),English,21/09/2016,https://www4.unfccc.int/sites/ndcstaging/Publi...


# 5. Exportación del dataset

Exportamos el dataset obtenido tanto en un fichero csv como en un fichero excel.

In [12]:
from datetime import date

today = date.today()
current_date = today.strftime("%d%m%Y")

In [13]:
df.to_excel('NDC_URL_' + current_date + '.xlsx', index=False)

In [14]:
df.to_csv('NDC_URL_' + current_date + '.csv', index=False)